In [1]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import operator
import json, time, sys
from __future__ import division
from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client
def fmt_date(d):
    return d.strftime("%Y%m%d")
def repartition(pipeline):
    return pipeline.repartition(MaxPartitions).cache()

%pylab inline

MaxPartitions = sc.defaultParallelism * 4
StartTime = datetime.datetime.now()

# Change this to False for local use.
RUN_AS_TELEMETRY_JOB = True

# Configuration for general data that spans all Firefox versions. Roughly,
# there are 100mil release channel pings, 7mil beta, 400k aurora, and 180k
# nightly for the given time window.
#
# To not explode the analysis time, the fractions below are adjusted
# around having ~6mil users total for all channels, or ~3mil total for
# beta+.
GeneralTimeWindow = 14
ReleaseFraction = 0.1
BetaFraction = 0.4
NightlyAndAuroraFraction = 0.5

Populating the interactive namespace from numpy and matplotlib


In [2]:
###########################################################
# Helper function block for fetching and filtering pings. #
###########################################################
def union_pipelines(a, b):
    if a is None:
        return b
    return a + b

# Helper for scaling back 
class FeatureDate(object):
    def __init__(self, availDate, mergeDate):
        self.availDate = availDate
        self.mergeDate = mergeDate

        
    # If a feature has not been available for a given time window yet,
    # scale up the amount of samples we take to get an equivalent sample
    # size.
    def scaleToAvailability(self, startDate, endDate, availDate, fraction):
        assert availDate > startDate
        want_days = (endDate - startDate).days
        avail_days = (endDate - availDate).days
        fraction = min((fraction * want_days) / avail_days, 1)
        return availDate, fraction
    
    # Try to scale up/down a channel's ping amount based on how long a
    # feature has been available.
    def computeChannel(self, startDate, endDate, availDate, fraction):
        if availDate <= startDate:
            return {
                'build_id': (startDate, endDate),
                'fraction': fraction
            }
        
        newStartDate, newFraction = self.scaleToAvailability(
            startDate = startDate,
            endDate = endDate,
            availDate = availDate,
            fraction = fraction)
        return {
            'build_id': (newStartDate, endDate),
            'fraction': newFraction
        }
    
    def computeChannels(self, startDate, endDate):
        channels = {}
        channels['nightly'] = self.computeChannel(
            startDate = startDate,
            endDate = endDate,
            availDate = self.availDate,
            fraction = NightlyAndAuroraFraction)
        
        # If we haven't merged to aurora, return now.
        if (endDate - self.mergeDate).days < 1:
            return channels
        channels['aurora'] = self.computeChannel(
            startDate = startDate,
            endDate = endDate,
            availDate = self.mergeDate,
            fraction = NightlyAndAuroraFraction)
        
        # Guesstimate the merge date for Beta
        betaDate = self.mergeDate + datetime.timedelta(6 * 7 + 1)
        if (endDate - betaDate).days < 1:
            return channels
        channels['beta'] = self.computeChannel(
            startDate = startDate,
            endDate = endDate,
            availDate = betaDate,
            fraction = BetaFraction)
        
        # If we've probably released this feature, just select across all
        # sessions.
        relDate = betaDate + datetime.timedelta(6 * 7 + 1)
        if (endDate - relDate).days > 1:
            return None

        return channels
    
def BuildPingArgs(**kwargs):
    timeWindow = kwargs.pop('timeWindow', GeneralTimeWindow)
    channel = kwargs.pop('channel', None)
    feature = kwargs.pop('feature', None)
    
    # Since builds take a bit to disseminate, go back about 4 hour. This is a
    # completely made up number.
    limit = datetime.timedelta(0, 60 * 60 * 4)
    now = datetime.datetime.now()
    start = now - datetime.timedelta(timeWindow) - limit
    end = now - limit
        
    # Build channel properties.
    channels = None
    if channel is None and feature is not None:
        channels = feature.computeChannels(start, end)
    elif channel is not None:
        # Use the old algorithm of build_id.
        channels = {
            channel: {
                'range': (start, end),
                'fraction': kwargs.pop('fraction'),
                'build_id': (start, end),
            }
        }
    
    # If we don't have any channel info, we select across all pings. Here we
    # use the submission date rather than build_id, since most pings will be
    # from the release channel where builds are infrequent.
    if channels is None:
        channels = {
            '*': {
                'submission_date': (start, end),
                'fraction': kwargs.pop('fraction', ReleaseFraction)
            }
        }

    baseArgs = {
        'app': 'Firefox',
        'schema': 'v4',
    }
    
    def fmt_dates(tup):
        return (fmt_date(tup[0]), fmt_date(tup[1]))
    def day_count(tup):
        return (tup[1] - tup[0]).days

    metadata = []
    for channel in channels:
        info = {}
        new_args = baseArgs.copy()
        
        if channel != '*':
            new_args['channel'] = channel
        info['channel'] = channel
        
        old_args = channels[channel]
        
        new_args['fraction'] = old_args['fraction']
        info['fraction'] = old_args['fraction']
            
        if 'build_id' in old_args:
            new_args['build_id'] = fmt_dates(old_args['build_id'])
            info['build_range'] = day_count(old_args['build_id'])
        if 'submission_date' in old_args:
            new_args['submission_date'] = fmt_dates(old_args['submission_date'])
            info['day_range'] = day_count(old_args['submission_date'])
        
        metadata.append({
            'args': new_args,
            'info': info,
        })  
    return metadata

def FetchRawPings(**kwargs):
    timestamp = datetime.datetime.now()
    metadata = BuildPingArgs(**kwargs)
    pings = None
    for props in metadata:
        subset = get_pings(sc, **props['args'])
        pings = union_pipelines(pings, subset)
    return pings, {
        'metadata': metadata,
        'timestamp': timestamp,
    }
        
# Transform each ping to make it easier to work with in later stages.
def Validate(p):
    name = p.get("environment/system/os/name") or 'w'
    version = p.get("environment/system/os/version") or '0'
    if name == 'Linux':
        p['OSVersion'] = None
        p['OS'] = 'Linux'
        p['OSName'] = 'Linux'
    elif name == 'Windows_NT':
        spmaj = p.get("environment/system/os/servicePackMajor") or '0'
        p['OSVersion'] = version + '.' + str(spmaj)
        p['OS'] = 'Windows-' + version + '.' + str(spmaj)
        p['OSName'] = 'Windows'
    elif name == 'Darwin':
        p['OSVersion'] = version
        p['OS'] = 'Darwin-' + version
        p['OSName'] = 'Darwin'
    else:
        p['OSVersion'] = version
        p['OS'] = '{0}-{1}'.format(name, version)
        p['OSName'] = name
    
    # Telemetry data isn't guaranteed to be well-formed so unfortunately
    # we have to do some validation on it. If we get to the end, we set
    # p['valid'] to True, and this gets filtered over later. In addition
    # we have a wrapper below to help fetch strings that may be null.
    if not p.get("environment/build/version", None):
        return p
    p['FxVersion'] = p["environment/build/version"].split('.')[0]
    
    # Verify that we have at least one adapter.
    try:
        adapter = p["environment/system/gfx/adapters"][0]
    except:
        return p
    if adapter is None or not hasattr(adapter, '__getitem__'):
        return p
    
    def T(obj, key):
        return obj.get(key, None) or 'Unknown'
    
    # We store the device ID as a vendor/device string, because the device ID
    # alone is not enough to determine whether the key is unique.
    #
    # We also merge 'Intel Open Source Technology Center' with the device ID
    # that should be reported, 0x8086, for simplicity.
    vendorID = T(adapter, 'vendorID')
    if vendorID == u'Intel Open Source Technology Center':
        p['vendorID'] = u'0x8086'
    else:
        p['vendorID'] = vendorID
    p['deviceID'] = u'{0}/{1}'.format(p['vendorID'], T(adapter, 'deviceID'))
    p['driverVersion'] = u'{0}/{1}'.format(p['vendorID'], T(adapter, 'driverVersion'))
    p['deviceAndDriver'] = u'{0}/{1}'.format(p['deviceID'], T(adapter, 'driverVersion'))
        
    p['valid'] = True
    return p

def reduce_pings(pings):
    return get_pings_properties(pings, [
      'clientId',
      "creationDate",
      "environment/settings/userPrefs",
      "environment/build/architecture",
      "environment/build/version",
      "environment/build/buildId",
      "environment/system/memoryMB",
      "environment/system/isWow64",
      "environment/system/cpu",
      "environment/system/os/name",
      "environment/system/os/version",
      "environment/system/os/servicePackMajor",
      "environment/system/os/servicePackMinor",
      "environment/system/gfx/adapters",
      "environment/system/gfx/features",
      "environment/system/gfx/monitors",
      "payload/histograms/DEVICE_RESET_REASON",
      "payload/histograms/GRAPHICS_SANITY_TEST",
      "payload/histograms/GRAPHICS_SANITY_TEST_REASON",
      "payload/histograms/GRAPHICS_DRIVER_STARTUP_TEST",
      "payload/info/revision",
    ])

def FormatPings(pings):
    pings = reduce_pings(pings)
    pings = get_one_ping_per_client(pings)
    pings = pings.map(Validate)
    filtered_pings = pings.filter(lambda p: p.get('valid', False) == True)
    return filtered_pings.cache()

def FetchAndFormat(**kwargs):
    raw_pings, info = FetchRawPings(**kwargs)
    return FormatPings(raw_pings), info

In [3]:
##################################################################
# Helper function block for massaging pings into aggregate data. #
##################################################################

# Take each key in |b| and add it to |a|, accumulating its value into
# |a| if it already exists.
def combiner(a, b):
    result = a
    for key in b:
        countA = a.get(key, 0)
        countB = b[key]
        result[key] = countA + countB
    return result

# Same as combiner(), but does not mutate either input.
def safe_combiner(left, right):
    result = left.copy()
    for key in right:
        result[key] = result.get(key, 0) + right[key]
    return result

# Return an aggregation based on combiner.
def aggregation(data, fn):
    view = data.map(fn)
    return view.reduceByKey(combiner)

# Helper for reduceByKey.
def map_x_to_y(data, sourceKey, destKey):
    def extract(p):
        return (p[sourceKey], { p[destKey]: 1 })
    return aggregation(data, extract)

# Helper for reduceByKey => count.
def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p[sourceKey],)
    return data.map(extract).countByKey()

# After reduceByKey(combiner), we get a mapping like:
#  key => { variable => value }
#
# This function collapses 'variable' instances below a threshold into
# a catch-all identifier ('Other').
def coalesce_to_n_items(agg, max_items):
    obj = []
    for superkey, breakdown in agg:
        if len(breakdown) <= max_items:
            obj += [(superkey, breakdown)]
            continue
        items = sorted(breakdown.items(), key=lambda obj: obj[1], reverse=True)
        new_breakdown = {k: v for k, v in items[0:max_items]}
        total = 0
        for k, v in items[max_items:]:
            total += v
        if total:
            new_breakdown['Other'] = new_breakdown.get('Other', 0) + total
        obj += [(superkey, new_breakdown)]
    return obj

In [4]:
#############################
# Helper for writing files. #
#############################

def ApplyPingInfo(obj, **kwargs):
    if 'pings' not in kwargs:
        return
    
    pings, info = kwargs.pop('pings')
    
    # To make the sample source information more transparent, we include
    # the breakdown of Firefox channel numbers.
    if '__share' not in info:
        info['__share'] = map_x_to_count(pings, 'FxVersion')
    
    obj['sessions'] = {
        'count': pings.count(),
        'timestamp': time.mktime(info['timestamp'].timetuple()),
        'metadata': info['metadata'],
        'share': info['__share'],
    }
    
def Export(filename, obj):
    if RUN_AS_TELEMETRY_JOB:
        filename = os.path.join('output', filename)
    with open(filename, 'w') as fp:
        json.dump(obj, fp)
        
def TimedExport(filename, callback, **kwargs):
    start = datetime.datetime.now()
    
    obj = callback()
    ApplyPingInfo(obj, **kwargs)
    
    end = datetime.datetime.now()
    elapsed = end - start
    obj['phaseTime'] = elapsed.total_seconds()
    
    Export(filename, obj)
    export_time = datetime.datetime.now() - end
    
    print('Computed {0} in {1} seconds.'.format(filename, elapsed.total_seconds()))
    print('Exported {0} in {1} seconds.'.format(filename, export_time.total_seconds()))
    
# Profiler for debugging.
class Prof(object):
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.sout('Starting {0}... '.format(self.name))
        self.start = datetime.datetime.now()
        return None
    def __exit__(self, type, value, traceback):
        self.end = datetime.datetime.now()
        self.sout('... {0}: {1}s'.format(self.name, (self.end - self.start).total_seconds()))
    def sout(self, s):
        sys.stdout.write(s)
        sys.stdout.write('\n')
        sys.stdout.flush()

In [5]:
def quiet_logs(sc):
  logger = sc._jvm.org.apache.log4j
  logger.LogManager.getLogger("org").setLevel(logger.Level.ERROR)
  logger.LogManager.getLogger("akka").setLevel(logger.Level.ERROR)
quiet_logs(sc)

In [6]:
# Get a general ping sample across all Firefox channels.
with Prof("General pings") as px:
    GeneralPings, GeneralPingInfo  = FetchAndFormat()
    GeneralPings = GeneralPings.cache()

    # Windows gets some preferential breakdown treatment.
    WindowsPings = GeneralPings.filter(lambda p: p['OSName'] == 'Windows')
    WindowsPings = WindowsPings.cache()

Starting General pings... 
... General pings: 556.190044s


In [7]:
# Results by operating system.
if '__share' not in GeneralPingInfo:
    GeneralPingInfo['__share'] = map_x_to_count(GeneralPings, 'FxVersion')

def GetGeneralStatisticsForSubset(subset, windows_subset):
    OSShare = map_x_to_count(subset, 'OSName')

    # Results by Windows version.
    WindowsShare = map_x_to_count(windows_subset, 'OSVersion')

    # Top-level stats.
    VendorShare = map_x_to_count(subset, 'vendorID')
    
    return {
        'os': OSShare,
        'windows': WindowsShare,
        'vendors': VendorShare,
    }

def GetGeneralStatistics():
    obj = {}
    obj['devices'] = map_x_to_count(GeneralPings, 'deviceID')
    obj['drivers'] = map_x_to_count(GeneralPings, 'driverVersion')
    
    byFx = {}
    with Prof('general stats for all') as px:
        byFx['all'] = GetGeneralStatisticsForSubset(GeneralPings, WindowsPings)
    
    for key in GeneralPingInfo['__share']:
        subset = GeneralPings.filter(lambda p: p['FxVersion'] == key)
        windows = subset.filter(lambda p: p['OSName'] == 'Windows')
        subset = repartition(subset)
        windows = repartition(windows)
        with Prof('general stats for ' + key) as px:
            byFx[key] = GetGeneralStatisticsForSubset(subset, windows)
        
    obj['byFx'] = byFx
    return obj
        
TimedExport(filename = 'general-statistics.json',
            callback = GetGeneralStatistics,
            pings = (GeneralPings, GeneralPingInfo))

Starting general stats for all... 
... general stats for all: 420.359319s
Starting general stats for 39... 
... general stats for 39: 408.813486s
Starting general stats for 44... 
... general stats for 44: 407.544638s
Starting general stats for 45... 
... general stats for 45: 424.198493s
Starting general stats for 42... 
... general stats for 42: 428.049759s
Starting general stats for 43... 
... general stats for 43: 431.628391s
Starting general stats for 40... 
... general stats for 40: 430.354483s
Starting general stats for 41... 
... general stats for 41: 440.532465s
Computed general-statistics.json in 3822.545237 seconds.
Exported general-statistics.json in 0.03137 seconds.


In [8]:
def GetDriverStatistics():
    obj = {}
    obj['deviceAndDriver'] = map_x_to_count(GeneralPings,'deviceAndDriver')
    return obj

TimedExport(filename = 'device-statistics.json',
            callback = GetDriverStatistics,
            pings = (GeneralPings, GeneralPingInfo))

Computed device-statistics.json in 298.913387 seconds.
Exported device-statistics.json in 0.204228 seconds.


In [9]:
DeviceResetReasonKey = 'payload/histograms/DEVICE_RESET_REASON'

#############################
# Perform the TDR analysis. #
#############################
def GetTDRStatistics():
    NumTDRReasons = 8
    def ping_has_tdr_for(p, reason):
        return p[DeviceResetReasonKey][reason] > 0

    # Specialized version of map_x_to_y, for TDRs. We cast to int because for
    # some reason the values Spark returns do not serialize with JSON.
    def map_reason_to_vendor(p, reason, destKey):
        return (int(reason), { p[destKey]: int(p[DeviceResetReasonKey][reason]) })
    def map_vendor_to_reason(p, reason, destKey):
        return (p[destKey], { int(reason): int(p[DeviceResetReasonKey][reason]) })

    # Filter out pings that do not have any TDR data. We expect this to be a huge reduction
    # in the sample set, and the resulting partition count gets way off. We repartition
    # immediately for performance.
    TDRSubset = WindowsPings.filter(lambda p: p.get(DeviceResetReasonKey, None) is not None)
    TDRSubset = TDRSubset.repartition(MaxPartitions)
    TDRSubset = TDRSubset.cache()

    # Aggregate the device reset data.
    TDRResults = TDRSubset.map(lambda p: p[DeviceResetReasonKey]).reduce(lambda x, y: x + y)

    # For each TDR reason, get a list tuple of (reason, vendor => resetCount). Then
    # we combine these into a single series.
    reason_to_vendor_tuples = None
    vendor_to_reason_tuples = None
    for reason in xrange(1, NumTDRReasons):
        subset = TDRSubset.filter(lambda p: ping_has_tdr_for(p, reason))
        subset = subset.cache()

        tuples = subset.map(lambda p: map_reason_to_vendor(p, reason, 'vendorID'))
        reason_to_vendor_tuples = union_pipelines(reason_to_vendor_tuples, tuples)

        tuples = subset.map(lambda p: map_vendor_to_reason(p, reason, 'vendorID'))
        vendor_to_reason_tuples = union_pipelines(vendor_to_reason_tuples, tuples)

    TDRReasonToVendor = reason_to_vendor_tuples.reduceByKey(combiner, MaxPartitions)
    TDRVendorToReason = vendor_to_reason_tuples.reduceByKey(combiner, MaxPartitions)
    
    return {
        'tdrPings': TDRSubset.count(),
        'results': [int(value) for value in TDRResults],
        'reasonToVendor': TDRReasonToVendor.collect(),
        'vendorToReason': TDRVendorToReason.collect(),
    }
    
# Write TDR statistics.
TimedExport(filename = 'tdr-statistics.json',
            callback = GetTDRStatistics,
            pings = (WindowsPings, GeneralPingInfo))

Computed tdr-statistics.json in 547.236235 seconds.
Exported tdr-statistics.json in 0.003545 seconds.


In [10]:
##########################
# Get system statistics. #
##########################

ArchKey = 'environment/build/architecture'
CpuKey = 'environment/system/cpu'
MemoryKey = 'environment/system/memoryMB'

def GetBucketedMemory(pings): 
    def get_bucket(p):
        x = int(p / 1000)
        if x < 1:
            return 'less_1gb'
        if x <= 4:
            return x
        if x <= 8:
            return '4_to_8'
        if x <= 16:
            return '8_to_16'
        if x <= 32:
            return '16_to_32'
        return 'more_32'
    
    memory_rdd = pings.map(lambda p: p.get(MemoryKey, 0))
    memory_rdd = memory_rdd.filter(lambda p: p > 0)
    memory_rdd = memory_rdd.map(lambda p: (get_bucket(p),))
    memory_rdd = repartition(memory_rdd)
    return memory_rdd.countByKey()

def GetCpuFeatures(pings):
    cpuid_rdd = pings.map(lambda p: p.get(CpuKey, None))
    cpuid_rdd = cpuid_rdd.filter(lambda p: p is not None)
    cpuid_rdd = cpuid_rdd.map(lambda p: p.get('extensions', None))
    
    # Unfortunately, Firefox 39 had a bug where CPU features could be reported even
    # if they weren't present. To detect this we filter pings that have ARMv6 support
    # on x86/64.
    cpuid_rdd = cpuid_rdd.filter(lambda p: p is not None and 'hasARMv6' not in p)
    cpuid_rdd = repartition(cpuid_rdd)

    # Count before we blow up the list.
    with Prof('cpu count for x86') as px:
        total = cpuid_rdd.count()

    cpuid_rdd = cpuid_rdd.flatMap(lambda p: [(ex, 1) for ex in p])
    with Prof('cpu features for x86') as px:
        feature_map = cpuid_rdd.countByKey()
        
    return {
        'total': total,
        'features': feature_map,
    }

def GetSystemStatistics():
    def get_logical_cores(p):
        cpu = p.get(CpuKey, None)
        if cpu is None:
            return 'unknown'
        return cpu.get('count', 'unknown')
    with Prof('logical cores') as px:
        logical_cores = GeneralPings.map(lambda p: (get_logical_cores(p),)).countByKey()
    
    cpu_features = GetCpuFeatures(GeneralPings)
    
    with Prof('memory buckets') as px:
        memory = GetBucketedMemory(GeneralPings)
        
    def get_os_bits(p):
        arch = p.get(ArchKey, 'unknown')
        if arch == 'x86-64':
            return '64'
        if arch == 'x86':
            wow64 = p.get("environment/system/isWow64", False)
            if wow64:
                return '32_on_64'
            return '32'
        return 'unknown'
    
    with Prof('OS bit count') as px:
        os_bits = WindowsPings.map(lambda p: (get_os_bits(p),)).countByKey()
    
    return {
        'logical_cores': logical_cores,
        'x86': cpu_features,
        'memory': memory,
        'wow': os_bits,
    }

TimedExport(filename = 'system-statistics.json',
            callback = GetSystemStatistics,
            pings = (GeneralPings, GeneralPingInfo))

Starting logical cores... 
... logical cores: 146.183732s
Starting cpu count for x86... 
... cpu count for x86: 154.327415s
Starting cpu features for x86... 
... cpu features for x86: 80.069669s
Starting memory buckets... 
... memory buckets: 154.43035s
Starting OS bit count... 
... OS bit count: 199.891968s
Computed system-statistics.json in 882.528412 seconds.
Exported system-statistics.json in 0.000607 seconds.


In [ ]:
# Done with global pings.
WindowsPings = None
GeneralPings = None
GeneralPingInfo = None

In [ ]:
# Get pings with graphics features. This landed in roughly the 7-19-2015 nightly.
FeaturesKey = 'environment/system/gfx/features'
UserPrefsKey = 'environment/settings/userPrefs'

WindowsFeaturePings, WindowsFeaturePingsInfo = FetchAndFormat(
    feature = FeatureDate(
        availDate = datetime.datetime(2015, 7, 20),
        mergeDate = datetime.datetime(2015, 8, 10))
)

def windows_feature_filter(p):
    return p['OSName'] == 'Windows' and p.get(FeaturesKey) is not None

WindowsFeatures = WindowsFeaturePings.filter(windows_feature_filter)
WindowsFeatures = WindowsFeatures.cache()

In [ ]:
# Build graphics feature statistics.
def get_compositor(p):
    compositor = p[FeaturesKey].get('compositor', 'none')
    if compositor == 'none':
        userPrefs = p.get(UserPrefsKey, None)
        if userPrefs is not None:
            omtc = userPrefs.get('layers.offmainthreadcomposition.enabled', True)
            if omtc != True:
                compositor = 'disabled'
    return (compositor,)

def get_d3d11_status(p):
    d3d11 = p[FeaturesKey].get('d3d11', None)
    if not hasattr(d3d11, '__getitem__'):
        return 'unknown'
    status = d3d11.get('status', 'unknown')
    if status != 'available':
        return status
    if d3d11.get('warp', False) == True:
        return 'warp'
    return d3d11.get('version', 'unknown')

def get_warp_status(p):
    if 'blacklisted' not in p[FeaturesKey]['d3d11']:
        return 'unknown'
    if p[FeaturesKey]['d3d11']['blacklisted'] == True:
        return 'blacklist'
    return 'device failure'

def get_d2d_status(p):
    d2d = p[FeaturesKey].get('d2d', None)
    if not hasattr(d2d, '__getitem__'):
        return ('unknown',)
    status = d2d.get('status', 'unknown')
    if status != 'available':
        return (status,)
    return (d2d.get('version', 'unknown'),)

def has_working_d3d11(p):
    d3d11 = p[FeaturesKey].get('d3d11', None)
    if d3d11 is None:
        return False
    return d3d11.get('status') == 'available'

def get_texture_sharing_status(p):
    return (p[FeaturesKey]['d3d11'].get('textureSharing', 'unknown'),)

# We skip certain windows versions in detail lists since this phase is
# very expensive to compute.
ImportantWindowsVersions = (
    '5.1.2',
    '5.1.3',
    '6.0.2',
    '6.1.0',
    '6.1.1',
    '6.2.0',
    '6.3.0',
    '10.0.0',
)

def GetWindowsFeatures():
    WindowsCompositorMap = WindowsFeatures.map(get_compositor).countByKey()
    D3D11StatusMap = WindowsFeatures.map(lambda p: (get_d3d11_status(p),)).countByKey()
    D2DStatusMap = WindowsFeatures.map(get_d2d_status).countByKey()
    
    warp_pings = WindowsFeatures.filter(lambda p: get_d3d11_status(p) == 'warp')
    warp_pings = repartition(warp_pings)
    WarpStatusMap = warp_pings.map(lambda p: (get_warp_status(p),)).countByKey()

    TextureSharingMap = WindowsFeatures.filter(has_working_d3d11).map(get_texture_sharing_status).countByKey()

    # Now, build the same data except per version.
    feature_pings_by_os = map_x_to_count(WindowsFeatures, 'OSVersion')
    WindowsFeaturesByVersion = {}
    for os_version in feature_pings_by_os:
        if os_version not in ImportantWindowsVersions:
            continue
        subset = WindowsFeatures.filter(lambda p: p['OSVersion'] == os_version)
        subset = repartition(subset)
        
        results = {
            'count': subset.count(),
            'compositors': subset.map(get_compositor).countByKey(),
        }
        try:
            if int(os_version.split('.')[0]) >= 6:
                results['d3d11'] = subset.map(lambda p: (get_d3d11_status(p),)).countByKey()
                results['d2d'] = subset.map(get_d2d_status).countByKey()
                
                warp_pings = subset.filter(lambda p: get_d3d11_status(p) == 'warp')
                results['warp'] = warp_pings.map(lambda p: (get_warp_status(p),)).countByKey()
        except:
            pass
        finally:
            # Free resources.
            warp_pings = None
            subset = None
        WindowsFeaturesByVersion[os_version] = results
    
    return {
        'all': {
            'compositors': WindowsCompositorMap,
            'd3d11': D3D11StatusMap,
            'd2d': D2DStatusMap,
            'textureSharing': TextureSharingMap,
            'warp': WarpStatusMap,
        },
        'byVersion': WindowsFeaturesByVersion,
    }

TimedExport(filename = 'windows-features.json',
            callback = GetWindowsFeatures,
            pings = (WindowsFeatures, WindowsFeaturePingsInfo))

In [ ]:
WindowsFeatures = None
WindowsFeaturePings = None
WindowsFeaturePingsInfo = None

In [ ]:
# Sanity test and crash analysis setup.
# The sanity test was only available in Firefox 41 and higher, so currently
# our analyses grabs a different sample of pings to be more accurate.
#
# Once the sanity test makes it to beta we can use the main data set.
sanity_test_ping_pool, sanity_test_info = FetchAndFormat(
    feature = FeatureDate(
        availDate = datetime.datetime(2015, 6, 25),
        mergeDate = datetime.datetime(2015, 6, 29))
)

# Set up constants.
SANITY_TEST_PASSED = 0
SANITY_TEST_FAILED_RENDER = 1
SANITY_TEST_FAILED_VIDEO = 2
SANITY_TEST_CRASHED = 3
SANITY_TEST_TIMEDOUT = 4
SANITY_TEST_LAST_VALUE = 5
SANITY_TEST_REASON_FIRST_RUN = 0
SANITY_TEST_REASON_FIREFOX_CHANGED = 1
SANITY_TEST_REASON_DEVICE_CHANGED = 2
SANITY_TEST_REASON_DRIVER_CHANGED = 3
SANITY_TEST_REASON_LAST_VALUE = 4

SANITY_TEST = 'payload/histograms/GRAPHICS_SANITY_TEST'
SANITY_TEST_REASON = 'payload/histograms/GRAPHICS_SANITY_TEST_REASON'

In [ ]:
# Filter sanity test pings.
def ping_has_sanity_test(p):
    return p.get(SANITY_TEST, None) is not None

In [ ]:
#########################
# Sanity test analysis. #
#########################
def GetSanityTestsForSlice(filter_func, is_xp = False):
    sanity_test_pings = sanity_test_ping_pool.filter(filter_func)
    data = sanity_test_pings.filter(ping_has_sanity_test)
    
    # We don't want to fold FAILED_LAYERS and FAILED_VIDEO into the same
    # resultset, so we use this function to split them out.
    def get_sanity_test_result(p):
        if p[SANITY_TEST][SANITY_TEST_PASSED] > 0:
            return SANITY_TEST_PASSED
        if p[SANITY_TEST][SANITY_TEST_CRASHED] > 0:
            return SANITY_TEST_CRASHED
        if p[SANITY_TEST][SANITY_TEST_FAILED_RENDER] > 0:
            return SANITY_TEST_FAILED_RENDER
        if p[SANITY_TEST][SANITY_TEST_FAILED_VIDEO] > 0:
            return SANITY_TEST_FAILED_VIDEO
        return SANITY_TEST_LAST_VALUE
    
    # Aggregate the sanity test data.
    with Prof('initial map') as px:
        SanityTestResults = data.map(lambda p: (get_sanity_test_result(p),)).countByKey()

    with Prof('share resolve') as px:
        os_share = map_x_to_count(data, 'OSVersion')
        
    with Prof('ping_count') as px:
        sanity_test_count = data.count()
        ping_count = sanity_test_pings.count()

    sanity_test_by_vendor = None
    sanity_test_by_os = None
    sanity_test_by_device = None
    sanity_test_by_driver = None
    for value in xrange(SANITY_TEST_FAILED_RENDER, SANITY_TEST_LAST_VALUE):
        if is_xp and value == SANITY_TEST_FAILED_VIDEO:
            continue

        subset = data.filter(lambda p: get_sanity_test_result(p) == value)

        tuples = subset.map(lambda p: (value, { p['vendorID']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_vendor = union_pipelines(sanity_test_by_vendor, tuples)
    
        tuples = subset.map(lambda p: (value, { p['OS']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_os = union_pipelines(sanity_test_by_os, tuples)
    
        tuples = subset.map(lambda p: (value, { p['deviceID']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_device = union_pipelines(sanity_test_by_device, tuples)
    
        tuples = subset.map(lambda p: (value, { p['driverVersion']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_driver = union_pipelines(sanity_test_by_driver, tuples)
            
    sanity_test_by_vendor = repartition(sanity_test_by_vendor)
    sanity_test_by_os = repartition(sanity_test_by_os)
    sanity_test_by_device = repartition(sanity_test_by_device)
    sanity_test_by_driver = repartition(sanity_test_by_driver)
    
    with Prof('vendor resolve') as px:
        SanityTestByVendor = sanity_test_by_vendor.reduceByKey(combiner)
    with Prof('os resolve') as px:
        SanityTestByOS = sanity_test_by_os.reduceByKey(combiner)
    with Prof('device resolve') as px:
        SanityTestByDevice = sanity_test_by_device.reduceByKey(combiner)
    with Prof('driver resolve') as px:
        SanityTestByDriver = sanity_test_by_driver.reduceByKey(combiner)
        
    print('Partitions: {0},{1},{2},{3}'.format(
        SanityTestByVendor.getNumPartitions(),
        SanityTestByOS.getNumPartitions(),
        SanityTestByDevice.getNumPartitions(),
        SanityTestByDriver.getNumPartitions()))
        
    with Prof('vendor collect') as px:
        byVendor = SanityTestByVendor.collect()
    with Prof('os collect') as px:
        byOS = SanityTestByOS.collect()
    with Prof('device collect') as px:
        byDevice = SanityTestByDevice.collect()
    with Prof('driver collect') as px:
        byDriver = SanityTestByDriver.collect()
    
    return {
        'sanityTestPings': sanity_test_count,
        'totalPings': ping_count,
        'results': SanityTestResults,
        'byVendor': byVendor,
        'byOS': byOS,
        'byDevice': coalesce_to_n_items(byDevice, 10),
        'byDriver': coalesce_to_n_items(byDriver, 10),
        'windows': os_share,
    }

def GetSanityTests(): 
    obj = {}
    
    def windows_xp(p):
        return p['OSName'] == 'Windows' and p['OSVersion'].startswith('5.')
    obj['windowsXP'] = GetSanityTestsForSlice(windows_xp, is_xp = True)
    
    def windows_vista(p):
        return p['OSName'] == 'Windows' and not p['OSVersion'].startswith('5.')
    obj['windows'] = GetSanityTestsForSlice(windows_vista)
    
    return obj
    
# Write Sanity Test statistics.
TimedExport(filename = 'sanity-test-statistics.json',
            callback = GetSanityTests,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
#################################
# Build sanity test crash data. #
#################################
def make_crash_report(p):
    obj = {
        'os': {
            'name': p['environment/system/os/name'],
            'version': p.get('environment/system/os/version', None),
            'servicePack': None,
        },
        'adapter': p['environment/system/gfx/adapters'][0],
        'build': {
            'version': p['environment/build/version'],
            'id': p.get('environment/build/buildId', None),
            'revision': p.get('payload/info/revision', None),
        },
        'date': p.get('creationDate', None)
    }
    if obj['os']['name'] == 'Windows_NT':
        spmaj = p.get('environment/system/os/servicePackMajor', 0)
        spmin = p.get('environment/system/os/servicePackMinor', 0)        
        if spmaj:
            if spmin:
                obj['os']['servicePack'] = '{0}.{1}'.format(spmaj, spmin)
            else:
                obj['os']['servicePack'] = '{0}'.format(spmaj)
    return obj

In [ ]:
#############################
# Sanity test crash reports #
#############################
def GetSanityTestCrashes():
    def ping_has_sanity_test_crash(p):
        return p[SANITY_TEST][SANITY_TEST_CRASHED] > 0

    sanity_test_pings = sanity_test_ping_pool.filter(ping_has_sanity_test)
    sanity_test_pings = repartition(sanity_test_pings)
    sanity_test_crash_pings = sanity_test_pings.filter(ping_has_sanity_test_crash)
    reports = sanity_test_crash_pings.map(make_crash_report)
    return {
        'sanityTestPings': sanity_test_pings.count(),
        'reports': reports.collect(),
    }
    
TimedExport(filename = 'sanity-test-crash-reports.json',
            callback = GetSanityTestCrashes,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
STARTUP_TEST_KEY = 'payload/histograms/GRAPHICS_DRIVER_STARTUP_TEST'
STARTUP_OK = 0
STARTUP_ENV_CHANGED = 1
STARTUP_CRASHED = 2
STARTUP_ACCEL_DISABLED = 3
    
def GetStartupTests():
    startup_test_pings = sanity_test_ping_pool.filter(lambda p: p.get(STARTUP_TEST_KEY, None) is not None)
    startup_test_pings = startup_test_pings.repartition(MaxPartitions)
    startup_test_pings = startup_test_pings.cache()

    StartupTestResults = startup_test_pings.map(lambda p: p[STARTUP_TEST_KEY]).reduce(lambda x, y: x + y)

    startup_test_crashes = startup_test_pings.filter(lambda p: p[STARTUP_TEST_KEY][STARTUP_CRASHED] > 0)
    StartupTestCrashes = startup_test_crashes.map(make_crash_report)
    
    os_share = map_x_to_count(startup_test_pings, 'OS')
    
    return {
        'startupTestPings': startup_test_pings.count(),
        'results': [int(i) for i in StartupTestResults],
        'reports': StartupTestCrashes.collect(),
        'windows': os_share,
    }
    
# Write startup test results.
TimedExport(filename = 'startup-test-statistics.json',
            callback = GetStartupTests,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
MonitorsKey = 'environment/system/gfx/monitors'

def get_monitor_count(p):
    monitors = p.get(MonitorsKey, None)
    try:
        return len(monitors)
    except:
        return 0
    
def get_monitor_res(p, i):
    width = p[MonitorsKey][i].get('screenWidth', 0)
    height = p[MonitorsKey][i].get('screenHeight', 0)
    if width == 0 or height == 0:
        return 'Unknown'
    return '{0}x{1}'.format(width, height)

In [ ]:
def GetMonitorStatistics():
    def get_monitor_rdds_for_index(data, i):
        def get_refresh_rate(p):
            refreshRate = p[MonitorsKey][i].get('refreshRate', 0)
            return refreshRate if refreshRate > 1 else 'Unknown'
        def get_resolution(p):
            return get_monitor_res(p, i)
    
        monitors_at_index = data.filter(lambda p: get_monitor_count(p) == monitor_count)
        monitors_at_index = repartition(monitors_at_index)
        refresh_rates = monitors_at_index.map(lambda p: (get_refresh_rate(p),))
        resolutions = monitors_at_index.map(lambda p: (get_resolution(p),))
        return refresh_rates, resolutions
    
    MonitorCounts = sanity_test_ping_pool.map(lambda p: (get_monitor_count(p),)).countByKey()
    MonitorCounts.pop(0, None)

    refresh_rates = None
    resolutions = None
    for monitor_count in MonitorCounts:
        rate_subset, res_subset = get_monitor_rdds_for_index(sanity_test_ping_pool, monitor_count - 1)
        refresh_rates = union_pipelines(refresh_rates, rate_subset)
        resolutions = union_pipelines(resolutions, res_subset)
    
    MonitorRefreshRates = refresh_rates.countByKey()
    MonitorResolutions = resolutions.countByKey()
    
    return {
        'counts': MonitorCounts,
        'refreshRates': MonitorRefreshRates,
        'resolutions': MonitorResolutions,
    }

TimedExport(filename = 'monitor-statistics.json',
            callback = GetMonitorStatistics,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
def GetAPZStatistics(pings):
    def apz_enabled(p):
        userPrefs = p.get(UserPrefsKey)
        if userPrefs is None:
            return True
        return userPrefs.get('layers.async-pan-zoom.enabled', True)
    
    apz_disabled = pings.filter(lambda p: not apz_enabled(p))
    apz_disabled = repartition(apz_disabled)
    byOS = apz_disabled.map(lambda p: (p['OS'],)).countByKey()
    byDevice = apz_disabled.map(lambda p: (p['deviceID'],)).countByKey()
    
    has_monitor = apz_disabled.filter(lambda p: get_monitor_count(p) > 0)
    byResolution = has_monitor.map(lambda p: (get_monitor_res(p, 0),)).countByKey()
    
    return {
        'disabled': apz_disabled.count(),
        'byOS': byOS,
        'byDevice': byDevice,
        'byResolution': byResolution,
    }

def ExportAPZStatistics():
    pings, info = FetchAndFormat(channel = 'nightly', fraction = 1.0)
    TimedExport(filename = 'apz-statistics.json',
                callback = lambda: GetAPZStatistics(pings),
                pings = (pings, info))
    
ExportAPZStatistics()

In [ ]:
EndTime = datetime.datetime.now()
TotalElapsed = (EndTime - StartTime).total_seconds()

print('Total time: {0}'.format(TotalElapsed))